<a href="https://colab.research.google.com/github/suleymanbalaban3/Estimating-Credit-Repayment-Potential-of-Companies/blob/master/logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#https://medium.com/deep-learning-turkiye/google-colab-ile-%C3%BCcretsiz-gpu-kullan%C4%B1m%C4%B1-30fdb7dd822e
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
#https://medium.com/deep-learning-turkiye/google-colab-ile-%C3%BCcretsiz-gpu-kullan%C4%B1m%C4%B1-30fdb7dd822e
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from google.colab import drive
drive.mount("/googledrive")

In [0]:
%mkdir /datathon4
%cp "drive/Datathon/Data.csv" /datathon4

cp: cannot stat 'drive/Datathon/Data.csv': No such file or directory


In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov  9 21:03:34 2018

@author: suleyman
"""
import csv
import pandas as pd
import math
import random
import pickle
from sklearn import linear_model
from scipy.spatial import distance

#LogisticRegression : Yi = C1*X1+ C2*X2 + .. + Cn*Xn
#C values will find and using for prediction

#Read data from csv file and fill values into two list (xValues, yValues)
def readData(inputFileName, xValues, yValues):
    i = 0
    with open(inputFileName, encoding="utf8") as csvfile:
        readCSV = csv.reader(csvfile, delimiter=';')
        for rows in readCSV:
            if i > 0:                                       #avoiding feature names
                j = 0
                xValuesOneRow = []
                yValuesOneRow = []
                for row in rows:
                    if j == 1:                              #taking y values (0 or 1)
                        yValuesOneRow.append(float(row))
                    else:
                        if row == '':
                            xValuesOneRow.append(float(0))  #if x value not exists, add new feature (0) 
                            xValuesOneRow.append(float(0))
                        else:
                            xValuesOneRow.append(float(1))  #if x value exists, add new feature (1) 
                            val = float(row)
                            if val == 0:                    #log10 for zero
                                xValuesOneRow.append(float(0))
                            elif val < 0:                   #log10 for negative numbers
                                val = val * -1
                                val = math.log10(val)
                                val = val * -1
                                xValuesOneRow.append(val)
                            else:                           #log10 for positive numbers
                                xValuesOneRow.append(math.log10(float(row)))
                    j += 1
                xValues.append(xValuesOneRow)               #append one transaction features into 2d list
                yValues.append(yValuesOneRow)               #append one transaction's y values into 2d list
            i +=1

#Generating average vector for each class Suspicious(0) or Non-Suspicious(1)
def generateAverageTransactionForEachGroup(xValues, yValues, suspiciousTransactionsAverage, nonSuspiciousTransactionsAverage):
    j = 0
    suspiciousTransactionsAverageSize = 0
    nonSuspiciousTransactionsAverageSize = 0
    
    for row in xValues:
        i = 0
        if 1.0 in yValues[j]:
            isSuspicious = True
            suspiciousTransactionsAverageSize += 1
        else:
            isSuspicious = False
            nonSuspiciousTransactionsAverageSize += 1
        for value in row:
            if i > 1:                                       #avoid to include primary key for summation
                if isSuspicious == True:
                    if suspiciousTransactionsAverageSize == 1:
                        suspiciousTransactionsAverage.append(value)
                    else:
                        suspiciousTransactionsAverage[i-2] += value
                else:
                    if nonSuspiciousTransactionsAverageSize == 1:
                        nonSuspiciousTransactionsAverage.append(value)
                    else:
                        nonSuspiciousTransactionsAverage[i-2] += value
            i += 1
        j += 1
        
    i = 0
    while i < len(suspiciousTransactionsAverage):           #divide the suspicious transactions summation to size
        suspiciousTransactionsAverage[i] /= suspiciousTransactionsAverageSize
        i += 1
        
    i = 0
    while i < len(nonSuspiciousTransactionsAverage):        #divide the non-suspicious transactions summation to size
        nonSuspiciousTransactionsAverage[i] /= nonSuspiciousTransactionsAverageSize
        i += 1

#Cosine similarity function
def calculateCosineSimilarity(transaction1, transaction2):
    dotProductValue = dotProduct(transaction1, transaction2)
    down = downValue(transaction1, transaction2)
    
    if down == 0:
        return 1- dotProductValue
    else:
        return 1 - (dotProductValue / down)
    
#Dot product function
def dotProduct(transaction1, transaction2):
    result = 0
    i = 0
    for values in transaction1:
        result += values * transaction2[i]
        i += 1
    return result
#Down value for cosine distance
#sqrt(sum(pow(transaction1[i],2))) * sqrt(sum(pow(transaction2[i],2)))
def downValue(transaction1, transaction2):
    resultTransaction1 = 0
    resultTransaction2 = 0
    i = 0
    for values in transaction1:
        resultTransaction1 += math.pow(values, 2)
        resultTransaction2 += math.pow(transaction2[i], 2)
        i += 1
    if resultTransaction1 < 0:
        resultTransaction1 = resultTransaction1 * -1
        resultTransaction1 = math.sqrt(resultTransaction1)
        resultTransaction1 = resultTransaction1 * -1
    if resultTransaction2 < 0:
        resultTransaction2 = resultTransaction2 * -1
        resultTransaction2 = math.sqrt(resultTransaction2)
        resultTransaction2 = resultTransaction2 * -1
    
    return resultTransaction1 * resultTransaction2

#Calculating Euclidean distance for two transactions
def calculateEuclideanDistance(transaction1, transaction2):
    return distance.euclidean(transaction1, transaction2)

#Adding new four fearure for each transaction
def addDistanceFeaturesToTransactions(xValues, yValues):        
    i = 0
    suspiciousTransactionsAverage = []
    nonSuspiciousTransactionsAverage = []
    
    generateAverageTransactionForEachGroup(xValues, yValues, suspiciousTransactionsAverage, nonSuspiciousTransactionsAverage)
    
    for row in xValues:                             #generate four new feature for each transaction
        cosineSimilaritySuspicious = calculateCosineSimilarity(row[2:], suspiciousTransactionsAverage)
        cosineSimilarityNonSuspicious = calculateCosineSimilarity(row[2:], nonSuspiciousTransactionsAverage)
        euclideanDistanceSuspicious = calculateEuclideanDistance(row[2:], suspiciousTransactionsAverage)
        euclideanDistanceNonSuspicious = calculateEuclideanDistance(row[2:], nonSuspiciousTransactionsAverage)
                                                    #adding new features
        xValues[i].append(cosineSimilaritySuspicious)
        xValues[i].append(cosineSimilarityNonSuspicious)
        xValues[i].append(euclideanDistanceSuspicious)
        xValues[i].append(euclideanDistanceNonSuspicious)
        
        i += 1
        
#Calculating real score not class (0 or 1)
def predicitonWithScoring(coefficients, xValuesForPrediction):
    i = 0
    result = 0
    for xValue in xValuesForPrediction:             #multiply coefficients and x values than sum
        result += coefficients[0][i] * xValue;
        i += 1
    return result

#Taking data file name with path
def takeFileNameFromConsole(fileNameFor):
    inputFileName = input('Can you please enter a file name with path for ' + fileNameFor + ' : ')
    return inputFileName

#Taking file name with path for writing results, this is optional
def takeFileNameToPrintResults():
    fileNameToSaveResults = ''
    check = input('Do you want to save result to a specific file Y/N :')
    if check == 'Y' or check == 'y':
        fileNameToSaveResults = input('Please enter file name with path for outputs :')
    return fileNameToSaveResults

#Taking file name with path for saving and loading model
def takeFileNameToSaveAndLoadModel():    
    fileNameToSaveAndLoadModel = input('Can you please enter file name with path for model saving and loading :')
    return fileNameToSaveAndLoadModel

#Writing results into file, this is optional
def printResultsToFile(fileNameToSaveResults, results):
    headers = ['Primary Key', 'Score']

    with open(fileNameToSaveResults, 'w') as filehandle:  
        filehandle.writelines("%s    " % header for header in headers)
        filehandle.writelines("\n")
        for result in results:
            filehandle.writelines("%s   " % round(result[0]))
            filehandle.writelines("         %s\n" % result[1])
    filehandle.close()

def prepareData(inputFileName, inputFileFor, xValues, yValues):
    print("\nReading " + inputFileName + " file for " + inputFileFor)
    
    readData(inputFileName, xValues, yValues)   #read data file
    
    allDataForShuffle = list(zip(xValues, yValues)) #shuffle via random
    random.shuffle(allDataForShuffle)
    xValues, yValues = zip(*allDataForShuffle)
    print("\nTransactions(" + inputFileFor + ")" + " shuffled via random")
    
    print("\nAdding extra, new features into" + inputFileFor +" transaction information...\n")
    addDistanceFeaturesToTransactions(xValues, yValues)
    
def trainAndTestModel(inputFileNameTrain, inputFileNameTest):
    xValuesTest = []
    yValuesTest = []    
    yValuesForTest = []
    xValuesTrain = []
    yValuesTrain = []    
    yValuesForTrain = []
    
    transactionsResults = []
    isLoadedModel = False
    
    fileNameToSaveResults = takeFileNameToPrintResults()
    fileNameToSaveAndLoadModel = takeFileNameToSaveAndLoadModel()
    
    if fileNameToSaveAndLoadModel != '':            #if model exists, ask for loading
        try:
            with open(fileNameToSaveAndLoadModel, 'rb') as file:  
                pickle_model = pickle.load(file)
            checkToLoadModel = input('Do you want to load trained(saved) model Y/N :')
            if checkToLoadModel == 'Y' or checkToLoadModel == 'y':
                isLoadedModel = True
        except:
            isLoadedModel = False
       
    print("\n______________________ Program Informations ___________________")
    prepareData(inputFileNameTrain, "training", xValuesTrain, yValuesTrain)
    prepareData(inputFileNameTest, "testing", xValuesTest, yValuesTest)
    
    logisticRegression = linear_model.LogisticRegression()
    
    for value in yValuesTrain:                      #y values preparing for training input format
        yValuesForTrain.append(value[0])
    for value in yValuesTest:                       #y values preparing for testing input format
        yValuesForTest.append(value[0])
        
    if isLoadedModel == True:                       
        print("\nSaved model loaded\n")
        logisticRegression = pickle_model
    else:
        print("\nModel is being trained. This will take more than 4 minutes..\n")    
        logisticRegression.fit(xValuesTrain, yValuesForTrain)
    
    i=0
    for xValue in xValuesTest:
        yPredictionScore = []
        score = predicitonWithScoring(logisticRegression.coef_, xValue)
        yPredictionScore.append(10 ** xValue[1])    #primary key inverse of log10
        yPredictionScore.append(score)              #prediction score
        transactionsResults.append(yPredictionScore)#2d list for visualization
        i += 1
    
    transactionsResults = sorted(transactionsResults, key=lambda x: x[0])
    
    dataFrame = pd.DataFrame(transactionsResults, columns = ['  Primary Key', '          Score'])
    print(dataFrame)
    
    accuracyScoreTrain = logisticRegression.score(xValuesTrain, yValuesForTrain) * 100
    accuracyScoreTest = logisticRegression.score(xValuesTest, yValuesForTest) * 100                                        
    print("\nTraining Accuracy Score : %" + str(accuracyScoreTrain))
    print("\nTesting Accuracy Score : %" + str(accuracyScoreTest))
    
    if fileNameToSaveResults != '':
        printResultsToFile(fileNameToSaveResults, transactionsResults)
        print("\n\nResults wrote to output file")
    
    if fileNameToSaveAndLoadModel != '':
        with open(fileNameToSaveAndLoadModel, 'wb') as file:  
            pickle.dump(logisticRegression, file)
        print("\nTrained model saved")
    
    print("\n------------------------ END OF PROGRAM --------------------------")        
def main():  
    print("----------------------- Start of Program -------------------------\n")
    print("__________________________ User Inputs ________________________\n")
    inputFileNameTrain = takeFileNameFromConsole("training")
    inputFileNameTest = takeFileNameFromConsole("testing")
    trainAndTestModel(inputFileNameTrain, inputFileNameTest)
    
     
if __name__ == "__main__":
    main()
#------------------------------ End of Program---------------------------------    